# Cassandra

## Conexión con Cassandra y borrado de datos

In [ ]:
%load_ext cql

In [ ]:
%%cql
DROP KEYSPACE black;

In [ ]:
%%cql
CREATE KEYSPACE black 
WITH replication = {'class':'SimpleStrategy', 'replication_factor': 1};

In [ ]:
%cql USE black;

## Creacción de las tablas

En **Cassandra** vamos a crear 2 tablas principales, una donde almacenamos los movimientos en formato desnormalizado, y otra que nos servirá para almacenar los importes acumulados por cliente.

También tenemos una tabla auxiliar, creada gracias a la funcionalidad que nos da la base de datos de VISTAS MATERIALIZADAS, que vamos a utilizar para facilitar otros patrones distintos de acceso a la información.

En la tabla que almacena los movimientos vamos a tener como **PARTITION_KEY** el campo nombre, ya que nos interesa tener en el mismo nodo todos los datos de una persona. Como **CUSTERING_KEY** vamos a tener el importe, debido a que en nuestro caso de uso queremos los datos ordenador por importe

<br><br> 

<img src="images/Modelo_Cassandra.png" width="800" height="500">

<br><br> 

In [ ]:
%%cql 
CREATE TABLE acum_movimientos_nombre (
    nombre          text,
    importe         counter,
    PRIMARY KEY (nombre)
)

In [ ]:
%%cql 
CREATE TABLE movimientos (
    fecha           date,
    hora            int,
    minuto          int,
    importe         decimal,
    comercio        text,
    actividad_completa text,
    actividad       text,
    nombre          text,
    funcion         text,
    organizacion    text,
    PRIMARY KEY ((nombre), importe, fecha, hora, minuto)
)
WITH CLUSTERING ORDER BY (importe DESC);

En la vista materializada no es necesario insertar datos ya que se ocupa la propia base de datos

In [ ]:
%%cql
CREATE MATERIALIZED VIEW vm_movimientos_by_actividad AS
   SELECT * FROM movimientos
   WHERE actividad IS NOT NULL 
         and importe IS NOT NULL 
         and fecha IS NOT NULL
         and hora IS NOT NULL 
         and minuto IS NOT NULL 
         and nombre IS NOT NULL
   PRIMARY KEY (actividad, importe, fecha,hora,minuto,nombre)
   WITH CLUSTERING ORDER BY (importe desc)

## Carga de datos en pandas

Cargamos los datos en formato Excel y los desnormalizamos ...

In [ ]:
import pandas as pd
df_mov = pd.read_excel("./data/black.xlsx", sheet_name= "Movimientos", engine='openpyxl')
df_miembros = pd.read_excel("./data/black.xlsx", sheet_name= "Miembros", engine='openpyxl')
df = pd.merge(df_mov, df_miembros, on = ['id_miembro'], how = 'inner')
df.info()

### Carga de datos en Cassandra

In [ ]:
from cassandra.cluster import Cluster, BatchStatement, ConsistencyLevel
cluster = Cluster()
session = cluster.connect('black')

A destacar varios aspectos:
- El tipo de las fechas. Hay que dejarlo con el tipo **Date** de Python para no tener problemas (El tipo de la columna es también Date)
- Los nulos en Pandas son del tipo NumPy.nan, por lo hay convertirlos a None para que la inserción en Cassandra sea correcta
- En la tabla de importes acumulados guardamos el importe en un dato de tipo **counter**, que tiene un tipo entero, por lo que multiplicamos el dato por 100 para poder almacenarlo correctamete

In [ ]:
df.loc[df.comercio.isnull(),['comercio']] = None
df.loc[df.organizacion.isnull(),['organizacion']] = None
df.loc[df.actividad.isnull(),['actividad']] = None
df.loc[df.actividad_completa.isnull(),['actividad_completa']] = None

In [ ]:
import dateutil

def insert_movientos(df):
    
    sql_insert = """
INSERT INTO movimientos (
importe,
actividad,
fecha,
actividad_completa,
comercio,
funcion,
hora,
minuto,
organizacion,
nombre
) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
"""

    for index in df.index:
        fecha = df['fecha'][index]
        
        data = [
            df["importe"][index],
            df["actividad"][index],
            fecha.date(),
            df["actividad_completa"][index],
            df["comercio"][index],
            df["funcion"][index],
            df["hora"][index],
            df["minuto"][index],
            df["organizacion"][index],
            df["nombre"][index],
        ]
        
        session.execute(sql_insert, data)
        
        importe_int = int(round(df["importe"][index] * 100))        
        session.execute("UPDATE acum_movimientos_nombre SET importe = importe + %s WHERE nombre = %s", 
                        [importe_int, 
                         df["nombre"][index]]
        )

Ya podemos insertar los datos ...

In [ ]:
insert_movientos(df)

## Consulta de datos

Función de utilidad que realiza un query en Cassandra y devuelve un DataFrame de Pandas

In [ ]:
def execute_query(sql):
    rows = session.execute(sql)
    return pd.DataFrame(list(rows))

### Los 10 movimientos mas caros

Para resolver esta query con la base de datos tendríamos que tener un PARTITION KEY única para todos los registros

In [ ]:
sql = """
SELECT nombre, fecha, actividad_completa, importe
FROM MOVIMIENTOS
"""
df = execute_query(sql)

In [ ]:
df \
    .sort_values('importe', ascending = False) \
    .head(10)

### Los movimientos de una persona concreta

Se resuelve la query con la base de datos, ya que la PARTITION KEY es el campo **nombre**, y los registros están ordenados por importe

In [ ]:
sql = """
select nombre, fecha, actividad_completa, importe
from movimientos
where nombre = 'Javier de Miguel Sánchez'
limit 10
"""
execute_query(sql)

### Las 10 personas que mas han gastado

Esta consulta se pueder resolver mediante el acumulado que habíamos creado ad-doc. Observa que el importe es entero, por lo que hay que dividirlo por 100 para convertirlo a decimal

In [ ]:
sql = """
select *
from acum_movimientos_nombre
"""
df = execute_query(sql)

In [ ]:
df \
    .assign(importe = df.importe / 100) \
    .sort_values('importe', ascending = False) \
    .head(10)

### Los 10 movimientos mas caros por actividad

Esta consulta se pueder resolver mediante la vista materializada que hemos creado ad-hoc para resolver esta consulta

In [ ]:
sql = """
select nombre, fecha, actividad, actividad_completa, importe
from vm_movimientos_by_actividad
where actividad = 'HOGAR'
limit 10
"""
execute_query(sql)